In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import polars as pl

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
import lightgbm as lgb
from sklearn.impute import SimpleImputer
import gc
from joblib import Parallel, delayed
from pathlib import Path
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time
import os
import pickle

In [2]:
SEMILLAS = [550007, 550019, 550031, 550033, 550047]

mes_train = 202103, 202102, 202101
mes_test = 202104
mes_kaggle = 202106
ganancia_acierto = 780000
costo_estimulo = 20000
# =====================

In [4]:
#Que archivos son? esos meses seleccionados SIN SLOPE
#df_train.write_csv("data/df_train_01_02_03.csv")
#df_test.write_csv("data/df_test_04.csv")   

In [5]:
#df_train = pd.read_csv("data/df_train_fe.csv")
#df_test = pd.read_csv("data/df_test_04.csv")

In [3]:
df_train = pd.read_csv("data/df_train_fe_sinslope.csv")

MemoryError: Unable to allocate 14.9 MiB for an array with shape (4, 486791) and data type int64

In [ ]:
df_test = pd.read_csv("data/df_test_fe_sinslope.csv")

In [5]:
df_train.shape

(486791, 803)

In [6]:
df_test.shape

(163418, 803)

In [10]:
# df_train_03_continua = pd.read_csv("data/df_train_01_02_03.csv")
# #Selecciono solo las 202103 en foto_es y continua en clase_ternaria
# df_train_03_continua = df_train_03_continua[(df_train_03_continua['foto_mes'] == 202103) & (df_train_03_continua['clase_ternaria'] == 'CONTINUA')]   

In [11]:
#guardar en csv
#df_train_03_continua.to_csv("data/df_train_03_continua.csv", index=False)

In [12]:
# Filtrar los meses deseados
# df_train = df_trainc


# #Guardar en CSV
# df_train.to_csv("df_train_c12.csv", index=False)


In [7]:
# Liberar memoria antes de empezar
gc.collect()

0

In [ ]:
df_kaggle = pd.read_csv("data/df_kaggle_fe_sinslope.csv")

In [ ]:
df_test.shape


(163418, 801)

In [ ]:
df_train.shape


(197886, 801)

In [ ]:
df_kaggle.shape

(164313, 801)

In [15]:
#tiene mi df_train la columna "clase_ternaria"?
print("clase_ternaria" in df_train.columns)

True


In [29]:
#Downsampling de la clase Continua en df_train
# ===== DOWNSAMPLING EN DF_TRAIN (ANTES DEL SPLIT) =====
# Identificar índices de cada clase
idx_continua = df_train[df_train['clase_ternaria'] == 'CONTINUA'].index
idx_no_continua = df_train[df_train['clase_ternaria'] != 'CONTINUA'].index

# Definir qué porcentaje de CONTINUA querés mantener
porcentaje_continua = 0.40  # 🔥 Mantener 20 % de CONTINUA

# Calcular cuántos samples mantener
n_samples_continua = int(len(idx_continua) * porcentaje_continua)

# Hacer el subsample aleatorio de CONTINUA
np.random.seed(42)
idx_continua_subsampled = np.random.choice(idx_continua, size=n_samples_continua, replace=False)

# Combinar índices
idx_df_balanced = np.concatenate([idx_continua_subsampled, idx_no_continua])

# Aplicar el subsample a df_train
df_train = df_train.loc[idx_df_balanced]

print(f"Dataset después de downsampling: {df_train.shape}")
print(f"CONTINUA: {(df_train['clase_ternaria']=='CONTINUA').sum()}")
print(f"NO CONTINUA: {(df_train['clase_ternaria']!='CONTINUA').sum()}")
# =======================================================


Dataset después de downsampling: (197886, 801)
CONTINUA: 192602
NO CONTINUA: 5284


In [34]:
# Liberar memoria antes de empezar
gc.collect()

0

##Optimización con LightGBM

In [35]:
df_train['clase_peso'] = 1.0

df_train.loc[df_train['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_train.loc[df_train['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Separar X e y
X = df_train.drop(["clase_ternaria", "clase_peso"], axis=1)  # ✅ Sacamos también clase_peso
y = df_train["clase_ternaria"]
pesos = df_train["clase_peso"]  # ✅ Guardamos los pesos

# Binarizar y
y_binaria = (y != "CONTINUA").astype(int)

# Split 70/30 (ahora incluimos los pesos)
X_train, X_val, y_train, y_val, pesos_train, pesos_val = train_test_split(
    X, y_binaria, pesos,  # ✅ Separamos X, y Y pesos
    train_size=0.7,
    random_state=42,
    stratify=y_binaria
)

print("Train:", X_train.shape, y_train.shape, pesos_train.shape)
print("Validation:", X_val.shape, y_val.shape, pesos_val.shape)

# Ahora en el Dataset:
train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=pesos_train  # ✅ Usamos los pesos del train
                          )

Train: (71109, 802) (71109,) (71109,)
Validation: (146038, 802) (146038,) (146038,)


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Separar X e y
X = df_train.drop(["clase_ternaria", "clase_peso"], axis=1)  # ✅ Sacamos también clase_peso
y = df_train["clase_ternaria"]
pesos = df_train["clase_peso"]  # ✅ Guardamos los pesos

# Binarizar y
y_binaria = (y != "CONTINUA").astype(int)

# Split 70/30 (ahora incluimos los pesos)
X_train, X_val, y_train, y_val, pesos_train, pesos_val = train_test_split(
    X, y_binaria, pesos,  # ✅ Separamos X, y Y pesos
    train_size=0.7,
    random_state=42,
    stratify=y_binaria
)

print("Train:", X_train.shape, y_train.shape, pesos_train.shape)
print("Validation:", X_val.shape, y_val.shape, pesos_val.shape)

# Ahora en el Dataset:
train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=pesos_train  # ✅ Usamos los pesos del train
                          )

In [11]:
def ganancia_prob(y_pred, data):
  weight = data.get_weight()
  ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
  ganancia = ganancia[np.argsort(y_pred)[::-1]]
  ganancia = np.cumsum(ganancia)  # ✅ Bien
  return 'gan_eval', np.max(ganancia), True
  


In [21]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 300)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.1) # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 50, 1500)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': SEMILLAS[0],
        'verbose': -1
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train, # eligir la clase
                              weight=pesos_train
                              )
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior
        #early_stopping_rounds= int(50 + 5 / learning_rate),
        feval=ganancia_prob,
        stratified=True,
        nfold=4,
        seed=SEMILLAS[0],
        callbacks=[
                lgb.early_stopping(stopping_rounds=int(50 + 5/learning_rate), verbose=False),
                lgb.log_evaluation(period=200),
                ]
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan 
# Al final de objective()
gc.collect()

#guardar el archivo en mi carpeta data
storage_name = "sqlite:///data/optimization_lgbm.db"

study_name = "exp_1_lgbm"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2025-10-31 16:48:31,608] Using an existing study with name 'exp_1_lgbm' instead of creating a new one.


In [22]:
study.optimize(objective, n_trials=10) # subir subir

[200]	valid's gan_eval: 3.12915e+08 + 1.32316e+07
[400]	valid's gan_eval: 3.1683e+08 + 1.34441e+07
[600]	valid's gan_eval: 3.1726e+08 + 1.34812e+07


[I 2025-10-31 16:50:37,835] Trial 2 finished with value: 318155000.0 and parameters: {'num_leaves': 239, 'learning_rate': 0.09675632362583497, 'min_data_in_leaf': 343, 'feature_fraction': 0.14845557131157736, 'bagging_fraction': 0.9603963995110338}. Best is trial 0 with value: 1287080000.0.


[200]	valid's gan_eval: 3.1969e+08 + 1.63468e+07


[W 2025-10-31 16:52:36,198] Trial 3 failed with parameters: {'num_leaves': 177, 'learning_rate': 0.09864195605252882, 'min_data_in_leaf': 423, 'feature_fraction': 0.6519206961918165, 'bagging_fraction': 0.2050592505151917} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Flor\Documents\UBA\DMEyF\venv\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Flor\AppData\Local\Temp\ipykernel_6464\1685824414.py", line 29, in objective
    cv_results = lgb.cv(
                 ^^^^^^^
  File "c:\Users\Flor\Documents\UBA\DMEyF\venv\Lib\site-packages\lightgbm\engine.py", line 829, in cv
    cvbooster.update(fobj=fobj)  # type: ignore[call-arg]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Flor\Documents\UBA\DMEyF\venv\Lib\site-packages\lightgbm\engine.py", line 417, in handler_function
    ret.append(getattr(booster, name)(*args, **kwarg

KeyboardInterrupt: 

In [7]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================
# CARGAR ESTUDIO DE OPTUNA
# ============================================================

print("="*60)
print("CARGANDO ESTUDIO DE OPTUNA")
print("="*60)

storage_name = "sqlite:///data/optimization_lgbm.db"
study_name = "exp_301_lgbm"

study = optuna.load_study(
    study_name=study_name,
    storage=storage_name
)

print(f"Estudio cargado: {study_name}")
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor valor: ${study.best_value:,.0f}")
print(f"Mejores parámetros: {study.best_params}")

# ============================================================
# OPCIÓN 1: GRÁFICOS INTERACTIVOS CON PLOTLY (OPTUNA)
# ============================================================

print("\n" + "="*60)
print("GENERANDO GRÁFICOS INTERACTIVOS (PLOTLY)")
print("="*60)

# 1. Historia de optimización
fig = plot_optimization_history(study)
fig.write_html("data/graficos/optimization_history.html")
print("✅ Guardado: data/graficos/optimization_history.html")

# 2. Importancia de parámetros
fig = plot_param_importances(study)
fig.write_html("data/graficos/param_importances.html")
print("✅ Guardado: data/graficos/param_importances.html")

# 3. Gráfico de contorno (relaciones entre pares de parámetros)
from optuna.visualization import plot_contour
fig = plot_contour(study, params=['num_leaves', 'learning_rate'])
fig.write_html("data/graficos/contour_leaves_lr.html")
print("✅ Guardado: data/graficos/contour_leaves_lr.html")

# 4. Slice plot (cada hiperparámetro vs ganancia)
from optuna.visualization import plot_slice
fig = plot_slice(study)
fig.write_html("data/graficos/slice_plot.html")
print("✅ Guardado: data/graficos/slice_plot.html")

# 5. Parallel coordinate plot
from optuna.visualization import plot_parallel_coordinate
fig = plot_parallel_coordinate(study)
fig.write_html("data/graficos/parallel_coordinate.html")
print("✅ Guardado: data/graficos/parallel_coordinate.html")

# ============================================================
# OPCIÓN 2: GRÁFICO

CARGANDO ESTUDIO DE OPTUNA
Estudio cargado: exp_301_lgbm
Número de trials: 894
Mejor valor: $1,785,980,000
Mejores parámetros: {'num_leaves': 147, 'learning_rate': 0.07055166350630575, 'min_data_in_leaf': 183, 'feature_fraction': 0.6992396968297147, 'bagging_fraction': 0.49046102060858093}

GENERANDO GRÁFICOS INTERACTIVOS (PLOTLY)
✅ Guardado: data/graficos/optimization_history.html
✅ Guardado: data/graficos/param_importances.html
✅ Guardado: data/graficos/contour_leaves_lr.html
✅ Guardado: data/graficos/slice_plot.html
✅ Guardado: data/graficos/parallel_coordinate.html


In [9]:
import json
# Obtener los mejores 10 trials
best_trials = study.trials_dataframe().sort_values('value', ascending=False).head(10)

# Opción 1: Guardar como lista de diccionarios (más completo)
mejores_params = []
for i, trial in enumerate(study.best_trials[:10] if len(study.best_trials) >= 10 else study.best_trials):
    mejores_params.append({
        'rank': i + 1,
        'trial_number': trial.number,
        'value': trial.value,
        'params': trial.params,
        'best_iter': trial.user_attrs.get('best_iter', None)
    })

# Guardar en JSON
with open('data/mejores_hiperparametros.json', 'w', encoding='utf-8') as f:
    json.dump(mejores_params, f, indent=4, ensure_ascii=False)



Modelo 1. Predecir abril

In [10]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import json
import matplotlib.pyplot as plt
import gc

# ============================================================
# PREPARAR DATOS DE TRAIN - OPTIMIZADO PARA MEMORIA
# ============================================================

print("="*60)
print("PREPARANDO DATOS TRAIN")
print("="*60)

# 2. Convertir a numpy inmediatamente (más eficiente en memoria)
print("Convirtiendo a numpy (float32)...")
X_train = df_train.drop(["clase_ternaria", "clase_peso"], axis=1).to_numpy().astype('float32')
y_train = df_train["clase_ternaria"].to_numpy()
pesos_train = df_train["clase_peso"].to_numpy().astype('float32')

# Binarizar
y_train_binaria = (y_train != "CONTINUA").astype('int8')

print(f"X_train: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train_binaria: {y_train_binaria.shape}, dtype: {y_train_binaria.dtype}")
print(f"Clase positiva: {y_train_binaria.sum():,}")
print(f"Clase negativa: {(y_train_binaria == 0).sum():,}")

# Liberar df_train de la memoria
print("Liberando df_train de memoria...")
del df_train, y_train
gc.collect()

print("✅ Datos TRAIN preparados y df_train liberado")

# ============================================================
# PREPARAR DATOS DE TEST
# ============================================================

print("\n" + "="*60)
print("PREPARANDO DATOS TEST")
print("="*60)

X_test = df_test.drop("clase_ternaria", axis=1).to_numpy().astype('float32')
y_test = df_test["clase_ternaria"].to_numpy()

print(f"X_test: {X_test.shape}, dtype: {X_test.dtype}")
print(f"y_test: {y_test.shape}")

# Liberar df_test si no lo necesitás más
print("Liberando df_test de memoria...")
del df_test
gc.collect()

print("✅ Datos TEST preparados")

# ============================================================
# DEFINIR MÉTRICA PERSONALIZADA DE GANANCIA
# ============================================================

ganancia_acierto = 780000
costo_estimulo = 20000

def ganancia_lgb(y_pred, data):
    """Métrica personalizada de ganancia para LightGBM."""
    weight = data.get_weight()
    
    # Calcular ganancia para cada predicción
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - costo_estimulo
    
    # Ordenar por predicción descendente
    indices = np.argsort(-y_pred)
    ganancia_ordenada = ganancia[indices]
    
    # Calcular ganancia acumulada máxima
    ganancia_acum = np.cumsum(ganancia_ordenada)
    max_ganancia = np.max(ganancia_acum)
    
    return 'ganancia', max_ganancia, True

# ============================================================
# ENTRENAR CON TRAIN Y EVALUAR EN TEST
# ============================================================

def entrenar_modelo_test(mejores_params_path='data/mejores_hiperparametros_reg.json'):
    """Entrena el modelo con df_train usando la métrica de ganancia."""
    
    print(f"\nCargando hiperparámetros desde {mejores_params_path}...")
    with open(mejores_params_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    config = data[0] if isinstance(data, list) else data
    
    print(f"Trial {config['trial_number']}")
    print(f"Ganancia esperada en CV: ${config['value']:,.0f}")
    
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': config['params']['num_leaves'],
        'learning_rate': config['params']['learning_rate'],
        'min_data_in_leaf': 147,
        'feature_fraction': config['params']['feature_fraction'],
        'bagging_fraction': config['params']['bagging_fraction'],
        'seed': SEMILLAS[0],
        'verbose': 1
    }
    
    print("\nParámetros del modelo:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    
    # Crear dataset (free_raw_data=True para ahorrar memoria)
    print(f"\nCreando dataset con X_train: {X_train.shape}")
    train_data = lgb.Dataset(X_train,
                             label=y_train_binaria,
                             weight=pesos_train,
                             free_raw_data=False)  # Cambiar a True si sigue fallando
    
    # Entrenar modelo
    print(f"Entrenando modelo con {config['best_iter']} iteraciones...")
    model = lgb.train(
        params,
        train_data,
        num_boost_round=config['best_iter'],
        feval=ganancia_lgb,
        callbacks=[lgb.log_evaluation(period=50)]
    )
    
    print("✅ Modelo entrenado exitosamente")
    
    # Liberar train_data
    del train_data
    gc.collect()
    
    return model, config


def calcular_ganancia_test(model, X_test, y_test):
    """Calcula la ganancia en TEST y encuentra el umbral óptimo."""
    
    print("\n" + "="*60)
    print("EVALUANDO EN TEST")
    print("="*60)
    
    print(f"Generando predicciones en X_test: {X_test.shape}")
    y_test_pred_prob = model.predict(X_test)
    
    indices_ordenados = np.argsort(-y_test_pred_prob)
    y_test_sorted = y_test[indices_ordenados]
    
    print("Calculando ganancia acumulada...")
    ganancia_cum = np.cumsum([
        (ganancia_acierto if y == "BAJA+2" else -costo_estimulo)
        for y in y_test_sorted
    ])
    
    max_ganancia_idx = np.argmax(ganancia_cum)
    max_ganancia = ganancia_cum[max_ganancia_idx]
    n_envios_optimo = max_ganancia_idx + 1
    threshold_optimo = y_test_pred_prob[indices_ordenados[max_ganancia_idx]]
    
    print(f"\n📊 RESULTADOS EN TEST:")
    print(f"  Ganancia máxima: ${max_ganancia:,.0f}")
    print(f"  N° de envíos óptimo: {n_envios_optimo:,}")
    print(f"  Threshold óptimo: {threshold_optimo:.6f}")
    print(f"  % de la base: {n_envios_optimo / len(y_test) * 100:.2f}%")
    
    print("\n📈 GANANCIA POR CANTIDAD DE ENVÍOS:")
    for percentil in [0.01, 0.025, 0.05, 0.10]:
        n = int(len(y_test) * percentil)
        ganancia_n = ganancia_cum[n-1] if n > 0 else 0
        print(f"  Top {percentil*100:.1f}% ({n:,} envíos): ${ganancia_n:,.0f}")
    
    n_baja2_total = (y_test == "BAJA+2").sum()
    n_baja2_capturados = (y_test_sorted[:n_envios_optimo] == "BAJA+2").sum()
    print(f"\n🎯 CAPTURA DE BAJA+2:")
    print(f"  Total BAJA+2: {n_baja2_total:,}")
    print(f"  Capturados: {n_baja2_capturados:,}")
    print(f"  Tasa: {n_baja2_capturados / n_baja2_total * 100:.2f}%")
    
    return {
        'ganancia_maxima': max_ganancia,
        'n_envios_optimo': n_envios_optimo,
        'threshold_optimo': threshold_optimo,
        'ganancia_acumulada': ganancia_cum,
        'probabilidades_ordenadas': y_test_pred_prob[indices_ordenados],
        'y_test_sorted': y_test_sorted,
        'n_baja2_capturados': n_baja2_capturados,
        'n_baja2_total': n_baja2_total
    }


# ============================================================
# FLUJO COMPLETO
# ============================================================

print("\n" + "="*60)
print("ENTRENAMIENTO CON TRAIN")
print("="*60)

model_test, config = entrenar_modelo_test('data/mejores_hiperparametros.json')

resultados_test = calcular_ganancia_test(model_test, X_test, y_test)

with open('data/resultados_test.json', 'w') as f:
    json.dump({
        'ganancia_maxima': float(resultados_test['ganancia_maxima']),
        'n_envios_optimo': int(resultados_test['n_envios_optimo']),
        'threshold_optimo': float(resultados_test['threshold_optimo']),
        'n_baja2_capturados': int(resultados_test['n_baja2_capturados']),
        'n_baja2_total': int(resultados_test['n_baja2_total'])
    }, f, indent=4)

print("\n✅ Resultados guardados")

model_test.save_model('data/modelo_train_test2.txt')
print("✅ Modelo guardado")

PREPARANDO DATOS TRAIN
Convirtiendo a numpy (float32)...
X_train: (101585, 802), dtype: float32
y_train_binaria: (101585,), dtype: int8
Clase positiva: 5,284
Clase negativa: 96,301
Liberando df_train de memoria...
✅ Datos TRAIN preparados y df_train liberado

PREPARANDO DATOS TEST
X_test: (163418, 802), dtype: float32
y_test: (163418,)
Liberando df_test de memoria...
✅ Datos TEST preparados

ENTRENAMIENTO CON TRAIN

Cargando hiperparámetros desde data/mejores_hiperparametros.json...
Trial 672
Ganancia esperada en CV: $1,785,980,000

Parámetros del modelo:
  objective: binary
  metric: custom
  boosting_type: gbdt
  first_metric_only: True
  boost_from_average: True
  feature_pre_filter: False
  max_bin: 31
  num_leaves: 147
  learning_rate: 0.07055166350630575
  min_data_in_leaf: 147
  feature_fraction: 0.6992396968297147
  bagging_fraction: 0.49046102060858093
  seed: 550007
  verbose: 1

Creando dataset con X_train: (101585, 802)
Entrenando modelo con 737 iteraciones...
[LightGBM] [I

In [13]:
importances = model_test.feature_importance()
feature_names = X.columns  # tomamos los nombres desde tu dataset original

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df.head(60))


NameError: name 'X' is not defined

#Ganancia modelo clase 211 MARS sin nada
Ganancia modelo optimizaod acá, 331 mars

## Entrenar Modelos


In [ ]:
#Modelo entrenado sólo con 1 2 y 3

In [15]:

import json 
# ============================================================
# CARGAR MODELO GUARDADO
# ============================================================

print("Cargando modelo guardado...")
model_test = lgb.Booster(model_file='data/modelo_train_test.txt')
print("✅ Modelo cargado exitosamente")

# ============================================================
# PREDECIR EN KAGGLE
# ============================================================

print("\n" + "="*60)
print("PREDICCIONES PARA KAGGLE")
print("="*60)

# Cargar el número óptimo de envíos
print("Cargando número óptimo de envíos...")
with open('data/resultados_test.json', 'r') as f:
    resultados_test = json.load(f)

n_envios_optimo = resultados_test['n_envios_optimo']
print(f"Número óptimo de envíos: {n_envios_optimo:,}")

# Preparar datos de Kaggle
print("\nPreparando datos de Kaggle...")
X_kaggle = df_kaggle.drop("clase_ternaria", axis=1).to_numpy().astype('float32')
clientes_kaggle = df_kaggle["numero_de_cliente"].to_numpy()

print(f"Datos de Kaggle: {X_kaggle.shape}")

# Predicciones probabilísticas
print("Generando predicciones...")
y_pred_prob = model_test.predict(X_kaggle)

# Seleccionar top N clientes
print(f"Seleccionando top {n_envios_optimo:,} clientes...")
indices_top = np.argsort(-y_pred_prob)[:n_envios_optimo]

# Crear predicción binaria
y_pred_bin = np.zeros(len(y_pred_prob), dtype=int)
y_pred_bin[indices_top] = 1

# Crear submission
submission = pd.DataFrame({
    "numero_de_cliente": clientes_kaggle,
    "Predicted": y_pred_bin
})


Cargando modelo guardado...
✅ Modelo cargado exitosamente

PREDICCIONES PARA KAGGLE
Cargando número óptimo de envíos...
Número óptimo de envíos: 10,712

Preparando datos de Kaggle...
Datos de Kaggle: (164313, 802)
Generando predicciones...
Seleccionando top 10,712 clientes...


In [ ]:

# Guardar
submission.to_csv("data/predicciones_kaggle_en_marz.csv", index=False)

print(f"\n✅ Archivo guardado: 'data/predicciones_kaggle.csv'")
print(f"Total registros: {len(submission):,}")
print(f"Predicciones positivas (1): {y_pred_bin.sum():,}")
print(f"Predicciones negativas (0): {(y_pred_bin == 0).sum():,}")

print("\n📄 Primeras filas:")
print(submission.head())


✅ Archivo guardado: 'data/predicciones_kaggle.csv'
Total registros: 164,313
Predicciones positivas (1): 10,712
Predicciones negativas (0): 153,601

📄 Primeras filas:
   numero_de_cliente  Predicted
0          433987585          0
1          434200463          0
2          434285203          0
3          434436672          0
4          434468833          0


## Modelo entrenado con 01..04

Modelo multiples semillas 

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import json
import gc
from datetime import datetime

# ============================================================
# PREPARAR DATOS: TRAIN + TEST COMBINADOS
# ============================================================

print("="*60)
print("PREPARANDO DATOS: TRAIN + TEST COMBINADOS")
print("="*60)

# 1. Crear pesos en df_train
print("Creando pesos en df_train...")
df_train['clase_peso'] = 1.0
df_train.loc[df_train['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_train.loc[df_train['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# # 2. Crear pesos en df_test
print("Creando pesos en df_test...")
df_test['clase_peso'] = 1.0
df_test.loc[df_test['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_test.loc[df_test['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# # 3. Concatenar df_train y df_test
print("Concatenando df_train y df_test...")
df_full = pd.concat([df_train, df_test], axis=0, ignore_index=True)

print(f"df_train: {df_train.shape}")
print(f"df_test: {df_test.shape}")
print(f"df_full: {df_full.shape}")

# 4. Convertir a numpy (optimizado) - SIN numero_de_cliente
print("\nConvirtiendo a numpy (float32)...")
X_full = df_full.drop(["clase_ternaria", "clase_peso", "numero_de_cliente"], axis=1).to_numpy().astype('float32')
y_full = df_full["clase_ternaria"].to_numpy()
pesos_full = df_full["clase_peso"].to_numpy().astype('float32')

# Binarizar
y_full_binaria = (y_full != "CONTINUA").astype('int8')

print(f"X_full: {X_full.shape}, dtype: {X_full.dtype}")
print(f"y_full_binaria: {y_full_binaria.shape}, dtype: {y_full_binaria.dtype}")
print(f"Clase positiva: {y_full_binaria.sum():,}")
print(f"Clase negativa: {(y_full_binaria == 0).sum():,}")

# Liberar memoria
print("\nLiberando df_train, df_test, df_full de memoria...")
del df_train, df_test, df_full, y_full
gc.collect()

print("✅ Datos preparados")

# ============================================================
# DEFINIR MÉTRICA PERSONALIZADA DE GANANCIA
# ============================================================

ganancia_acierto = 780000
costo_estimulo = 20000

def ganancia_lgb(y_pred, data):
    """Métrica personalizada de ganancia para LightGBM."""
    weight = data.get_weight()
    
    # Calcular ganancia para cada predicción
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - costo_estimulo
    
    # Ordenar por predicción descendente
    indices = np.argsort(-y_pred)
    ganancia_ordenada = ganancia[indices]
    
    # Calcular ganancia acumulada máxima
    ganancia_acum = np.cumsum(ganancia_ordenada)
    max_ganancia = np.max(ganancia_acum)
    
    return 'ganancia', max_ganancia, True

# ============================================================
# CARGAR HIPERPARÁMETROS
# ============================================================

print("\n" + "="*60)
print("CARGANDO HIPERPARÁMETROS")
print("="*60)

with open('data/mejores_hiperparametros2.json', 'r') as f:
    data = json.load(f)

config = data[0] if isinstance(data, list) else data

print(f"Trial {config['trial_number']}")
print(f"Iteraciones: {config['best_iter']}")

# ============================================================
# ENTRENAR UN MODELO POR SEMILLA
# ============================================================

print("\n" + "="*60)
print(f"ENTRENANDO {len(SEMILLAS)} MODELOS (UNO POR SEMILLA)")
print("="*60)

modelos = []

for i, semilla in enumerate(SEMILLAS):
    print(f"\n--- Modelo {i+1}/{len(SEMILLAS)} - Semilla: {semilla} ---")
    
    # Configurar parámetros con la semilla correspondiente
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': config['params']['num_leaves'],
        'learning_rate': config['params']['learning_rate'],
        'min_data_in_leaf': config['params']['min_data_in_leaf'],
        'feature_fraction': config['params']['feature_fraction'],
        'bagging_fraction': config['params']['bagging_fraction'],
        'seed': semilla,  # ✅ Cambiar semilla
        'verbose': -1  # Silencioso para no saturar output
    }
    
    # Crear dataset
    full_data = lgb.Dataset(X_full,
                            label=y_full_binaria,
                            weight=pesos_full)
    
    # Entrenar modelo
    print(f"Entrenando con {config['best_iter']} iteraciones...")
    model = lgb.train(
        params,
        full_data,
        num_boost_round=config['best_iter'],
        feval=ganancia_lgb,
        callbacks=[lgb.log_evaluation(period=0)]  # Sin logs
    )
    
    # Guardar modelo
    model.save_model(f'data/predicciones/modelo_semilla_{semilla}.txt')
    print(f"✅ Modelo guardado: 'data/predicciones/modelo_semilla_{semilla}.txt'")
    
    # Agregar a la lista
    modelos.append(model)
    
    # Liberar memoria
    del full_data
    gc.collect()

print(f"\n✅ {len(modelos)} modelos entrenados exitosamente")

# Liberar X_full de memoria
del X_full, y_full_binaria, pesos_full
gc.collect()

# ============================================================
# CARGAR DF_KAGGLE Y PREDECIR CON CADA MODELO
# ============================================================

print("\n" + "="*60)
print("CARGANDO DF_KAGGLE Y PREDICIENDO")
print("="*60)

# Leer df_kaggle con tipos optimizados
print("Leyendo df_kaggle...")
columnas = pd.read_csv("data/df_kaggle.csv", nrows=0).columns.tolist()

# Crear diccionario de tipos: float32 para todas EXCEPTO numero_de_cliente
dtypes = {col: 'float32' for col in columnas if col not in ['numero_de_cliente', 'clase_ternaria']}

df_kaggle = pd.read_csv(
    "data/df_kaggle.csv",
    dtype=dtypes,
    low_memory=True
)

print(f"df_kaggle: {df_kaggle.shape}")
print(f"Tipo de numero_de_cliente: {df_kaggle['numero_de_cliente'].dtype}")


# Preparar datos
clientes_kaggle = df_kaggle["numero_de_cliente"].values
X_kaggle = df_kaggle.drop(["clase_ternaria", "numero_de_cliente"], axis=1).to_numpy().astype('float32')

print(f"X_kaggle: {X_kaggle.shape}, dtype={X_kaggle.dtype}")

del df_kaggle
gc.collect()



PREPARANDO DATOS: TRAIN + TEST COMBINADOS
Creando pesos en df_train...
Creando pesos en df_test...
Concatenando df_train y df_test...
df_train: (197886, 802)
df_test: (163418, 802)
df_full: (361304, 802)

Convirtiendo a numpy (float32)...
X_full: (361304, 799), dtype: float32
y_full_binaria: (361304,), dtype: int8
Clase positiva: 7,369
Clase negativa: 353,935

Liberando df_train, df_test, df_full de memoria...
✅ Datos preparados

CARGANDO HIPERPARÁMETROS
Trial 218
Iteraciones: 98

ENTRENANDO 5 MODELOS (UNO POR SEMILLA)

--- Modelo 1/5 - Semilla: 550007 ---
Entrenando con 98 iteraciones...
✅ Modelo guardado: 'data/predicciones/modelo_semilla_550007.txt'

--- Modelo 2/5 - Semilla: 550019 ---
Entrenando con 98 iteraciones...
✅ Modelo guardado: 'data/predicciones/modelo_semilla_550019.txt'

--- Modelo 3/5 - Semilla: 550031 ---
Entrenando con 98 iteraciones...
✅ Modelo guardado: 'data/predicciones/modelo_semilla_550031.txt'

--- Modelo 4/5 - Semilla: 550033 ---
Entrenando con 98 iteraciones

NameError: name 'X_full' is not defined

In [37]:
df_kaggle = df_kaggle.drop(columns=['cprestamos_personales', 'mprestamos_personales'])

NameError: name 'df_kaggle' is not defined

In [ ]:

# ============================================================
# PREDECIR CON CADA MODELO Y PROMEDIAR
# ============================================================

print("\n" + "="*60)
print("GENERANDO PREDICCIONES CON CADA MODELO")
print("="*60)

# Inicializar array para acumular probabilidades
probabilidades_acumuladas = np.zeros(len(X_kaggle), dtype='float64')

for i, (model, semilla) in enumerate(zip(modelos, SEMILLAS)):
    print(f"\nModelo {i+1}/{len(modelos)} - Semilla {semilla}")
    print("Prediciendo...")
    
    y_pred_prob = model.predict(X_kaggle)
    
    print(f"  Prob min: {y_pred_prob.min():.6f}")
    print(f"  Prob max: {y_pred_prob.max():.6f}")
    print(f"  Prob media: {y_pred_prob.mean():.6f}")
    
    # Acumular probabilidades
    probabilidades_acumuladas += y_pred_prob
    
    # Liberar modelo de memoria
    del model
    gc.collect()

# Promediar probabilidades
print("\n" + "="*60)
print("PROMEDIANDO PROBABILIDADES")
print("="*60)

y_pred_prob_promedio = probabilidades_acumuladas / len(SEMILLAS)

print(f"Probabilidades promediadas:")
print(f"  Prob min: {y_pred_prob_promedio.min():.6f}")
print(f"  Prob max: {y_pred_prob_promedio.max():.6f}")
print(f"  Prob media: {y_pred_prob_promedio.mean():.6f}")

# Guardar probabilidades promediadas (opcional)
np.save('data/predicciones/probabilidades_promedio.npy', y_pred_prob_promedio)
print("\n✅ Probabilidades guardadas en 'data/predicciones/probabilidades_promedio.npy'")


In [17]:
train_cols = set(X_full.columns)
kaggle_cols = set(X_kaggle.columns)

print("Extra en kaggle:", kaggle_cols - train_cols)
print("Faltan en kaggle:", train_cols - kaggle_cols)


NameError: name 'X_full' is not defined

In [15]:

# ============================================================
# BINARIZAR CON THRESHOLD DE PROBABILIDAD
# ============================================================

print("\n" + "="*60)
print("BINARIZANDO PREDICCIONES")
print("="*60)

threshold = 0.030
print(f"Threshold de probabilidad: {threshold}")

# Binarizar según threshold
y_pred_bin = (y_pred_prob_promedio > threshold).astype('int8')

print(f"Predicciones positivas (prob > {threshold}): {y_pred_bin.sum():,}")
print(f"% del total: {y_pred_bin.sum() / len(y_pred_bin) * 100:.2f}%")

# ============================================================
# CREAR SUBMISSION
# ============================================================

print("\n" + "="*60)
print("CREANDO SUBMISSION")
print("="*60)

submission = pd.DataFrame({
    "numero_de_cliente": clientes_kaggle,
    "Predicted": y_pred_bin
})

# Verificar tipos
print(f"Tipos en submission:")
print(f"  numero_de_cliente: {submission['numero_de_cliente'].dtype}")
print(f"  Predicted: {submission['Predicted'].dtype}")

# Obtener fecha y hora actual en formato YYYYMMDD_HHMM
fecha = datetime.now().strftime("%Y%m%d_%H%M")

# Crear el nombre del archivo con fecha incluida
filename = f"data/predicciones_kaggle_ensemble_{len(SEMILLAS)}semillas_threshold{threshold}_{fecha}.csv"
submission.to_csv(filename, index=False)

print(f"\n✅ Archivo guardado: '{filename}'")
print(f"\n📊 RESUMEN:")
print(f"  Modelos entrenados: {len(SEMILLAS)}")
print(f"  Total registros: {len(submission):,}")
print(f"  Predicciones positivas (1): {y_pred_bin.sum():,}")
print(f"  Predicciones negativas (0): {(y_pred_bin == 0).sum():,}")
print(f"  % positivos: {y_pred_bin.sum() / len(submission) * 100:.2f}%")

print("\n📄 Primeras 10 filas:")
print(submission.head(10))

print("\nVerificación de IDs:")
print(f"  Primer ID: {submission['numero_de_cliente'].iloc[0]}")
print(f"  Último ID: {submission['numero_de_cliente'].iloc[-1]}")

print("\n🎉 Ensemble de modelos completado - Predicciones listas para Kaggle!")



BINARIZANDO PREDICCIONES
Threshold de probabilidad: 0.03
Predicciones positivas (prob > 0.03): 14,668
% del total: 8.93%

CREANDO SUBMISSION
Tipos en submission:
  numero_de_cliente: int64
  Predicted: int8

✅ Archivo guardado: 'data/predicciones_kaggle_ensemble_5semillas_threshold0.03_20251031_1732.csv'

📊 RESUMEN:
  Modelos entrenados: 5
  Total registros: 164,313
  Predicciones positivas (1): 14,668
  Predicciones negativas (0): 149,645
  % positivos: 8.93%

📄 Primeras 10 filas:
   numero_de_cliente  Predicted
0          433987585          0
1          434200463          1
2          434285203          0
3          434436672          0
4          434468833          1
5          434500404          0
6          434626792          0
7          434672369          0
8          434687467          0
9          434694905          0

Verificación de IDs:
  Primer ID: 433987585
  Último ID: 951048415

🎉 Ensemble de modelos completado - Predicciones listas para Kaggle!


## -4k era entrenado con 01..03 y predecir 06 con los mejores hiperparametros de mi primer corrida (30) con metric ganancia
## -3k simil anterior pero modelo reentrenado con 01..04 con metric AUC 
## -3 K simil anterior pero modelo reentrenado con 01..04 con metric custom
## -2 umbral fijo de 12k 
## 0 de 01 02 y 03 sólo los baja+1/2. 

## voy a probar con slopes. no me sirvió. solo con dl tampoco (aunque no optimicé hip para esto)

## 2 con ensamble de semillas
1 si dejo los continua de marzo
4 con hiperparametros optimizados 500. hip2
5 con num fijo de 12300. esto es con fe todo sin slopes
3 Optimizacion con Claude y regularizacion. NO FUNCIONO

In [ ]:
# ============================================================
# PARTE 2: ENTRENAMIENTO CON DATASETS con todos los continua
# ============================================================

print("\n" + "="*60)
print("="*60)
print("INICIANDO ENTRENAMIENTO CON DATASETS SLOPE")
print("="*60)
print("="*60)

# ============================================================
# CARGAR DATASETS SLOPE
# ============================================================

print("\n" + "="*60)
print("CARGANDO DATASETS SLOPE")
print("="*60)

df_train_slope = pd.read_csv("data/df_train_01_02_03.csv")
df_test_slope = pd.read_csv("data/df_test_04.csv")
df_kaggle_slope = pd.read_csv("data/df_kaggle.csv")

print(f"df_train_slope: {df_train_slope.shape}")
print(f"df_test_slope: {df_test_slope.shape}")
print(f"df_kaggle_slope: {df_kaggle_slope.shape}")

# ============================================================
# PREPARAR DATOS SLOPE: TRAIN + TEST COMBINADOS
# ============================================================

print("\n" + "="*60)
print("PREPARANDO DATOS SLOPE: TRAIN + TEST COMBINADOS")
print("="*60)

# 1. Crear pesos en df_train_slope
print("Creando pesos en df_train_slope...")
df_train_slope['clase_peso'] = 1.0
df_train_slope.loc[df_train_slope['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_train_slope.loc[df_train_slope['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# 2. Crear pesos en df_test_slope
print("Creando pesos en df_test_slope...")
df_test_slope['clase_peso'] = 1.0
df_test_slope.loc[df_test_slope['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_test_slope.loc[df_test_slope['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# 3. Concatenar df_train_slope y df_test_slope
print("Concatenando df_train_slope y df_test_slope...")
df_full_slope = pd.concat([df_train_slope, df_test_slope], axis=0, ignore_index=True)

print(f"df_train_slope: {df_train_slope.shape}")
print(f"df_test_slope: {df_test_slope.shape}")
print(f"df_full_slope: {df_full_slope.shape}")

# 4. Convertir a numpy (optimizado) - SIN numero_de_cliente
print("\nConvirtiendo a numpy (float32)...")
X_full_slope = df_full_slope.drop(["clase_ternaria", "clase_peso", "numero_de_cliente"], axis=1).to_numpy().astype('float32')
y_full_slope = df_full_slope["clase_ternaria"].to_numpy()
pesos_full_slope = df_full_slope["clase_peso"].to_numpy().astype('float32')

# Binarizar
y_full_slope_binaria = (y_full_slope != "CONTINUA").astype('int8')

print(f"X_full_slope: {X_full_slope.shape}, dtype: {X_full_slope.dtype}")
print(f"y_full_slope_binaria: {y_full_slope_binaria.shape}, dtype: {y_full_slope_binaria.dtype}")
print(f"Clase positiva: {y_full_slope_binaria.sum():,}")
print(f"Clase negativa: {(y_full_slope_binaria == 0).sum():,}")

# Liberar memoria
print("\nLiberando df_train_slope, df_test_slope, df_full_slope de memoria...")
del df_train_slope, df_test_slope, df_full_slope, y_full_slope
gc.collect()

print("✅ Datos SLOPE preparados")

# ============================================================
# ENTRENAR MODELOS SLOPE (UNO POR SEMILLA)
# ============================================================

print("\n" + "="*60)
print(f"ENTRENANDO {len(SEMILLAS)} MODELOS SLOPE (UNO POR SEMILLA)")
print("="*60)

modelos_slope = []

for i, semilla in enumerate(SEMILLAS):
    print(f"\n--- Modelo SLOPE {i+1}/{len(SEMILLAS)} - Semilla: {semilla} ---")
    
    # Configurar parámetros con la semilla correspondiente (mismos hiperparámetros)
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': config['params']['num_leaves'],
        'learning_rate': config['params']['learning_rate'],
        'min_data_in_leaf': config['params']['min_data_in_leaf'],
        'feature_fraction': config['params']['feature_fraction'],
        'bagging_fraction': config['params']['bagging_fraction'],
        'seed': semilla,
        'verbose': -1, 
        'is_unbalanced': True  # Manejar desbalanceo automáticamente
    }
    
    # Crear dataset
    full_data_slope = lgb.Dataset(X_full_slope,
                                   label=y_full_slope_binaria,
                                   weight=pesos_full_slope)
    
    # Entrenar modelo
    print(f"Entrenando con {config['best_iter']} iteraciones...")
    model_slope = lgb.train(
        params,
        full_data_slope,
        num_boost_round=config['best_iter'],
        feval=ganancia_lgb,
        callbacks=[lgb.log_evaluation(period=0)]
    )
    
    # Guardar modelo
    model_slope.save_model(f'data/predicciones/modelo_slope_semilla_{semilla}.txt')
    print(f"✅ Modelo SLOPE guardado: 'data/predicciones/modelo_slope_semilla_{semilla}.txt'")
    
    # Agregar a la lista
    modelos_slope.append(model_slope)
    
    # Liberar memoria
    del full_data_slope
    gc.collect()

print(f"\n✅ {len(modelos_slope)} modelos SLOPE entrenados exitosamente")

# Liberar X_full_slope de memoria
del X_full_slope, y_full_slope_binaria, pesos_full_slope
gc.collect()

# ============================================================
# CARGAR DF_KAGGLE_SLOPE Y PREDECIR
# ============================================================

print("\n" + "="*60)
print("CARGANDO DF_KAGGLE_SLOPE Y PREDICIENDO")
print("="*60)

# Leer df_kaggle_slope con tipos optimizados
print("Leyendo df_kaggle_slope...")
columnas_slope = pd.read_csv("data/df_kaggle.csv", nrows=0).columns.tolist()

# Crear diccionario de tipos: float32 para todas EXCEPTO numero_de_cliente
dtypes_slope = {col: 'float32' for col in columnas_slope if col not in ['numero_de_cliente', 'clase_ternaria']}

df_kaggle_slope = pd.read_csv(
    "data/df_kaggle.csv",
    dtype=dtypes_slope,
    low_memory=True
)

print(f"df_kaggle_slope: {df_kaggle_slope.shape}")
print(f"Tipo de numero_de_cliente: {df_kaggle_slope['numero_de_cliente'].dtype}")

# Preparar datos
clientes_kaggle_slope = df_kaggle_slope["numero_de_cliente"].values
X_kaggle_slope = df_kaggle_slope.drop(["clase_ternaria", "numero_de_cliente"], axis=1).to_numpy().astype('float32')

print(f"X_kaggle_slope: {X_kaggle_slope.shape}, dtype={X_kaggle_slope.dtype}")

del df_kaggle_slope
gc.collect()

# ============================================================
# PREDECIR CON CADA MODELO SLOPE Y PROMEDIAR
# ============================================================

print("\n" + "="*60)
print("GENERANDO PREDICCIONES SLOPE CON CADA MODELO")
print("="*60)

# Inicializar array para acumular probabilidades
probabilidades_acumuladas_slope = np.zeros(len(X_kaggle_slope), dtype='float64')

for i, (model_slope, semilla) in enumerate(zip(modelos_slope, SEMILLAS)):
    print(f"\nModelo SLOPE {i+1}/{len(modelos_slope)} - Semilla {semilla}")
    print("Prediciendo...")
    
    y_pred_prob_slope = model_slope.predict(X_kaggle_slope)
    
    print(f"  Prob min: {y_pred_prob_slope.min():.6f}")
    print(f"  Prob max: {y_pred_prob_slope.max():.6f}")
    print(f"  Prob media: {y_pred_prob_slope.mean():.6f}")
    
    # Acumular probabilidades
    probabilidades_acumuladas_slope += y_pred_prob_slope
    
    # Liberar modelo de memoria
    del model_slope
    gc.collect()

# Promediar probabilidades
print("\n" + "="*60)
print("PROMEDIANDO PROBABILIDADES SLOPE")
print("="*60)

y_pred_prob_promedio_slope = probabilidades_acumuladas_slope / len(SEMILLAS)

print(f"Probabilidades promediadas SLOPE:")
print(f"  Prob min: {y_pred_prob_promedio_slope.min():.6f}")
print(f"  Prob max: {y_pred_prob_promedio_slope.max():.6f}")
print(f"  Prob media: {y_pred_prob_promedio_slope.mean():.6f}")

# Guardar probabilidades promediadas
np.save('data/predicciones/probabilidades_promedio_slope.npy', y_pred_prob_promedio_slope)
print("\n✅ Probabilidades SLOPE guardadas en 'data/predicciones/probabilidades_promedio_slope.npy'")

# ============================================================
# BINARIZAR CON THRESHOLD DE PROBABILIDAD SLOPE
# ============================================================

print("\n" + "="*60)
print("BINARIZANDO PREDICCIONES SLOPE")
print("="*60)

threshold_slope = 0.025
print(f"Threshold de probabilidad SLOPE: {threshold_slope}")

# Binarizar según threshold
y_pred_bin_slope = (y_pred_prob_promedio_slope > threshold_slope).astype('int8')

print(f"Predicciones positivas (prob > {threshold_slope}): {y_pred_bin_slope.sum():,}")
print(f"% del total: {y_pred_bin_slope.sum() / len(y_pred_bin_slope) * 100:.2f}%")

# ============================================================
# CREAR SUBMISSION SLOPE
# ============================================================

print("\n" + "="*60)
print("CREANDO SUBMISSION SLOPE")
print("="*60)

submission_slope = pd.DataFrame({
    "numero_de_cliente": clientes_kaggle_slope,
    "Predicted": y_pred_bin_slope
})

# Verificar tipos
print(f"Tipos en submission SLOPE:")
print(f"  numero_de_cliente: {submission_slope['numero_de_cliente'].dtype}")
print(f"  Predicted: {submission_slope['Predicted'].dtype}")

# Obtener fecha y hora actual
fecha_slope = datetime.now().strftime("%Y%m%d_%H%M")

# Crear el nombre del archivo con fecha incluida
filename_slope = f"data/predicciones_kaggle_SLOPE_ensemble_{len(SEMILLAS)}semillas_threshold{threshold_slope}_{fecha_slope}.csv"
submission_slope.to_csv(filename_slope, index=False)

print(f"\n✅ Archivo SLOPE guardado: '{filename_slope}'")
print(f"\n📊 RESUMEN SLOPE:")
print(f"  Modelos entrenados: {len(SEMILLAS)}")
print(f"  Total registros: {len(submission_slope):,}")
print(f"  Predicciones positivas (1): {y_pred_bin_slope.sum():,}")
print(f"  Predicciones negativas (0): {(y_pred_bin_slope == 0).sum():,}")
print(f"  % positivos: {y_pred_bin_slope.sum() / len(submission_slope) * 100:.2f}%")

print("\n📄 Primeras 10 filas SLOPE:")
print(submission_slope.head(10))

print("\nVerificación de IDs SLOPE:")
print(f"  Primer ID: {submission_slope['numero_de_cliente'].iloc[0]}")
print(f"  Último ID: {submission_slope['numero_de_cliente'].iloc[-1]}")

print("\n🎉 Ensemble de modelos SLOPE completado - Predicciones listas para Kaggle!")


INICIANDO ENTRENAMIENTO CON DATASETS SLOPE

CARGANDO DATASETS SLOPE
df_train_slope: (486791, 803)
df_test_slope: (163418, 803)
df_kaggle_slope: (164313, 803)

PREPARANDO DATOS SLOPE: TRAIN + TEST COMBINADOS
Creando pesos en df_train_slope...
Creando pesos en df_test_slope...
Concatenando df_train_slope y df_test_slope...
df_train_slope: (486791, 804)
df_test_slope: (163418, 804)
df_full_slope: (650209, 804)

Convirtiendo a numpy (float32)...
X_full_slope: (650209, 801), dtype: float32
y_full_slope_binaria: (650209,), dtype: int8
Clase positiva: 7,369
Clase negativa: 642,840

Liberando df_train_slope, df_test_slope, df_full_slope de memoria...
✅ Datos SLOPE preparados

ENTRENANDO 5 MODELOS SLOPE (UNO POR SEMILLA)

--- Modelo SLOPE 1/5 - Semilla: 550007 ---
Entrenando con 98 iteraciones...
✅ Modelo SLOPE guardado: 'data/predicciones/modelo_slope_semilla_550007.txt'

--- Modelo SLOPE 2/5 - Semilla: 550019 ---
Entrenando con 98 iteraciones...
✅ Modelo SLOPE guardado: 'data/predicciones/mo

In [9]:
# ============================================================
# ENSAMBLE DE MODELOS: ESTÁNDAR + SLOPE
# ============================================================

print("\n" + "="*60)
print("="*60)
print("ENSAMBLE DE PREDICCIONES: ESTÁNDAR + SLOPE")
print("="*60)
print("="*60)

# ============================================================
# ALINEAR DATASETS POR NUMERO_DE_CLIENTE
# ============================================================

print("\n" + "="*60)
print("ALINEANDO DATASETS POR NUMERO_DE_CLIENTE")
print("="*60)

# Crear DataFrames con las predicciones
df_pred_estandar = pd.DataFrame({
    'numero_de_cliente': clientes_kaggle,
    'prob_estandar': y_pred_prob_promedio,
    'pred_estandar': y_pred_bin
})

df_pred_slope = pd.DataFrame({
    'numero_de_cliente': clientes_kaggle_slope,
    'prob_slope': y_pred_prob_promedio_slope,
    'pred_slope': y_pred_bin_slope
})

print(f"Registros modelo estándar: {len(df_pred_estandar):,}")
print(f"Registros modelo SLOPE: {len(df_pred_slope):,}")

# Hacer merge por numero_de_cliente
df_ensemble = pd.merge(
    df_pred_estandar,
    df_pred_slope,
    on='numero_de_cliente',
    how='inner'  # Solo clientes que están en ambos
)

print(f"\n✅ Registros después del merge: {len(df_ensemble):,}")

# Verificar si hay clientes que no matchean
clientes_solo_estandar = set(clientes_kaggle) - set(clientes_kaggle_slope)
clientes_solo_slope = set(clientes_kaggle_slope) - set(clientes_kaggle)

if clientes_solo_estandar:
    print(f"⚠️  Clientes solo en estándar: {len(clientes_solo_estandar):,}")
if clientes_solo_slope:
    print(f"⚠️  Clientes solo en SLOPE: {len(clientes_solo_slope):,}")

if len(df_ensemble) < len(df_pred_estandar):
    print(f"⚠️  Atención: Se perdieron {len(df_pred_estandar) - len(df_ensemble):,} registros en el merge")

# ============================================================
# MÉTODO 1: ENSAMBLE POR VOTACIÓN (UNIÓN)
# ============================================================

print("\n" + "="*60)
print("MÉTODO 1: ENSAMBLE POR VOTACIÓN (UNIÓN)")
print("="*60)

# Crear predicción por votación: 1 si CUALQUIERA de los dos modelos predice 1
df_ensemble['pred_votacion'] = (
    (df_ensemble['pred_estandar'] == 1) | (df_ensemble['pred_slope'] == 1)
).astype('int8')

print(f"\n📊 ESTADÍSTICAS DE VOTACIÓN:")
print(f"  Predicciones modelo estándar (1): {df_ensemble['pred_estandar'].sum():,}")
print(f"  Predicciones modelo SLOPE (1): {df_ensemble['pred_slope'].sum():,}")
print(f"  Predicciones UNIÓN (1): {df_ensemble['pred_votacion'].sum():,}")
print(f"  % del total: {df_ensemble['pred_votacion'].sum() / len(df_ensemble) * 100:.2f}%")

# Análisis de overlap
solo_estandar = ((df_ensemble['pred_estandar'] == 1) & (df_ensemble['pred_slope'] == 0)).sum()
solo_slope = ((df_ensemble['pred_estandar'] == 0) & (df_ensemble['pred_slope'] == 1)).sum()
ambos = ((df_ensemble['pred_estandar'] == 1) & (df_ensemble['pred_slope'] == 1)).sum()

print(f"\n🔍 ANÁLISIS DE OVERLAP:")
print(f"  Solo modelo estándar: {solo_estandar:,}")
print(f"  Solo modelo SLOPE: {solo_slope:,}")
print(f"  Ambos modelos: {ambos:,}")
print(f"  Jaccard similarity: {ambos / df_ensemble['pred_votacion'].sum():.2%}")

# Crear submission por votación
submission_votacion = df_ensemble[['numero_de_cliente', 'pred_votacion']].copy()
submission_votacion.columns = ['numero_de_cliente', 'Predicted']

# Guardar
fecha_votacion = datetime.now().strftime("%Y%m%d_%H%M")
filename_votacion = f"data/predicciones_kaggle_ENSEMBLE_VOTACION_{len(SEMILLAS)}semillas_{fecha_votacion}.csv"
submission_votacion.to_csv(filename_votacion, index=False)

print(f"\n✅ Archivo votación guardado: '{filename_votacion}'")

# ============================================================
# MÉTODO 2: ENSAMBLE POR PROMEDIO DE PROBABILIDADES
# ============================================================

print("\n" + "="*60)
print("MÉTODO 2: ENSAMBLE POR PROMEDIO DE PROBABILIDADES")
print("="*60)

# Promedio simple de ambas probabilidades
df_ensemble['prob_ensemble'] = (df_ensemble['prob_estandar'] + df_ensemble['prob_slope']) / 2

print(f"Probabilidades ensemble:")
print(f"  Prob min: {df_ensemble['prob_ensemble'].min():.6f}")
print(f"  Prob max: {df_ensemble['prob_ensemble'].max():.6f}")
print(f"  Prob media: {df_ensemble['prob_ensemble'].mean():.6f}")

# Guardar probabilidades ensemble
np.save('data/predicciones/probabilidades_ensemble.npy', df_ensemble['prob_ensemble'].values)
print("\n✅ Probabilidades ensemble guardadas")

# ============================================================
# OPCIÓN 2A: USAR THRESHOLD EN ENSEMBLE
# ============================================================

print("\n--- Opción 2A: Usar threshold en ensemble ---")

threshold_ensemble = 0.025
print(f"Threshold ensemble: {threshold_ensemble}")

# Binarizar según threshold
df_ensemble['pred_ensemble'] = (df_ensemble['prob_ensemble'] > threshold_ensemble).astype('int8')

print(f"Predicciones positivas: {df_ensemble['pred_ensemble'].sum():,}")
print(f"% del total: {df_ensemble['pred_ensemble'].sum() / len(df_ensemble) * 100:.2f}%")

# Crear submission ensemble
submission_ensemble = df_ensemble[['numero_de_cliente', 'pred_ensemble']].copy()
submission_ensemble.columns = ['numero_de_cliente', 'Predicted']

# Guardar
fecha_ensemble = datetime.now().strftime("%Y%m%d_%H%M")
filename_ensemble = f"data/predicciones_kaggle_ENSEMBLE_PROMEDIO_{len(SEMILLAS)}semillas_threshold{threshold_ensemble}_{fecha_ensemble}.csv"
submission_ensemble.to_csv(filename_ensemble, index=False)

print(f"✅ Archivo ensemble guardado: '{filename_ensemble}'")

# ============================================================
# OPCIÓN 2B: PROBAR DIFERENTES THRESHOLDS
# ============================================================

print("\n--- Opción 2B: Generar múltiples versiones con diferentes thresholds ---")

thresholds_a_probar = [0.015, 0.020, 0.025, 0.030, 0.035, 0.040]

for thresh in thresholds_a_probar:
    print(f"\nGenerando ensemble con threshold {thresh}...")
    
    df_temp = df_ensemble.copy()
    df_temp['pred_temp'] = (df_temp['prob_ensemble'] > thresh).astype('int8')
    
    n_positivos = df_temp['pred_temp'].sum()
    pct_positivos = n_positivos / len(df_temp) * 100
    
    submission_temp = df_temp[['numero_de_cliente', 'pred_temp']].copy()
    submission_temp.columns = ['numero_de_cliente', 'Predicted']
    
    filename_temp = f"data/predicciones/ensemble_promedio_threshold{thresh}.csv"
    submission_temp.to_csv(filename_temp, index=False)
    print(f"  ✅ Guardado: {filename_temp} ({n_positivos:,} positivos - {pct_positivos:.2f}%)")

# ============================================================
# RESUMEN COMPARATIVO DE TODOS LOS ENSAMBLES
# ============================================================

print("\n" + "="*60)
print("RESUMEN COMPARATIVO DE ENSAMBLES")
print("="*60)

resumen_df = pd.DataFrame({
    'Método': [
        'Modelo Estándar',
        'Modelo SLOPE',
        'Ensemble Votación (Unión)',
        'Ensemble Promedio',
    ],
    'N° Envíos': [
        df_ensemble['pred_estandar'].sum(),
        df_ensemble['pred_slope'].sum(),
        df_ensemble['pred_votacion'].sum(),
        df_ensemble['pred_ensemble'].sum(),
    ],
    '% Base': [
        f"{df_ensemble['pred_estandar'].sum() / len(df_ensemble) * 100:.2f}%",
        f"{df_ensemble['pred_slope'].sum() / len(df_ensemble) * 100:.2f}%",
        f"{df_ensemble['pred_votacion'].sum() / len(df_ensemble) * 100:.2f}%",
        f"{df_ensemble['pred_ensemble'].sum() / len(df_ensemble) * 100:.2f}%",
    ]
})

print("\n")
print(resumen_df.to_string(index=False))

# ============================================================
# ANÁLISIS DE CORRELACIÓN ENTRE PROBABILIDADES
# ============================================================

print("\n" + "="*60)
print("ANÁLISIS DE CORRELACIÓN")
print("="*60)

correlacion = df_ensemble['prob_estandar'].corr(df_ensemble['prob_slope'])
print(f"Correlación entre probabilidades (Estándar vs SLOPE): {correlacion:.4f}")

if correlacion > 0.95:
    print("  📌 Muy alta correlación - los modelos son muy similares")
elif correlacion > 0.85:
    print("  ✅ Alta correlación - modelos consistentes pero con diferencias")
elif correlacion > 0.70:
    print("  🔄 Correlación moderada - buena diversidad para ensemble")
else:
    print("  ⚠️  Correlación baja - modelos muy diferentes (verificar)")

# ============================================================
# VISUALIZACIÓN DE DIFERENCIAS (OPCIONAL)
# ============================================================

print("\n" + "="*60)
print("ANÁLISIS DE PREDICCIONES DIVERGENTES")
print("="*60)

# Calcular diferencias
df_ensemble['diff_prob'] = abs(df_ensemble['prob_estandar'] - df_ensemble['prob_slope'])

# Top 10 con mayor diferencia
top_divergentes = df_ensemble.nlargest(10, 'diff_prob')

print(f"\nTop 10 clientes con mayor divergencia en probabilidades:")
print(f"{'Cliente':<20} {'Prob Estándar':<15} {'Prob SLOPE':<15} {'Diferencia':<15}")
print("-" * 65)
for _, row in top_divergentes.iterrows():
    print(f"{row['numero_de_cliente']:<20} {row['prob_estandar']:<15.6f} {row['prob_slope']:<15.6f} {row['diff_prob']:<15.6f}")

# ============================================================
# GUARDAR ANÁLISIS COMPLETO
# ============================================================

print("\n" + "="*60)
print("GUARDANDO ANÁLISIS COMPLETO")
print("="*60)

analisis_completo = {
    'fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'semillas_utilizadas': SEMILLAS.tolist() if hasattr(SEMILLAS, 'tolist') else list(SEMILLAS),
    'registros_alineados': int(len(df_ensemble)),
    'correlacion_modelos': float(correlacion),
    'estadisticas': {
        'modelo_estandar': {
            'n_envios': int(df_ensemble['pred_estandar'].sum()),
            'porcentaje': float(df_ensemble['pred_estandar'].sum() / len(df_ensemble) * 100)
        },
        'modelo_slope': {
            'n_envios': int(df_ensemble['pred_slope'].sum()),
            'porcentaje': float(df_ensemble['pred_slope'].sum() / len(df_ensemble) * 100)
        },
        'ensemble_votacion': {
            'n_envios': int(df_ensemble['pred_votacion'].sum()),
            'porcentaje': float(df_ensemble['pred_votacion'].sum() / len(df_ensemble) * 100),
            'solo_estandar': int(solo_estandar),
            'solo_slope': int(solo_slope),
            'ambos': int(ambos)
        },
        'ensemble_promedio': {
            'n_envios': int(df_ensemble['pred_ensemble'].sum()),
            'porcentaje': float(df_ensemble['pred_ensemble'].sum() / len(df_ensemble) * 100)
        }
    },
    'archivos_generados': {
        'votacion': filename_votacion,
        'promedio': filename_ensemble,
        'estandar': filename,
        'slope': filename_slope
    }
}

with open('data/analisis_ensemble.json', 'w') as f:
    json.dump(analisis_completo, f, indent=4)

print("✅ Análisis guardado en 'data/analisis_ensemble.json'")

# ============================================================
# RESUMEN FINAL
# ============================================================

print("\n" + "="*60)
print("="*60)
print("🎉 PROCESO DE ENSEMBLE COMPLETADO")
print("="*60)
print("="*60)

print(f"\n📁 ARCHIVOS GENERADOS:")
print(f"  1️⃣  Modelo Estándar: {filename}")
print(f"  2️⃣  Modelo SLOPE: {filename_slope}")
print(f"  3️⃣  Ensemble Votación: {filename_votacion}")
print(f"  4️⃣  Ensemble Promedio: {filename_ensemble}")

print(f"\n💡 RECOMENDACIONES:")
if correlacion > 0.90:
    print("  • Alta correlación entre modelos - el ensemble por promedio puede ser mejor")
    print("  • Considera usar el ensemble por promedio")
else:
    print("  • Correlación moderada - ambos métodos de ensemble son válidos")
    print("  • Prueba ambos en Kaggle para ver cuál funciona mejor")

print(f"\n🎯 PRÓXIMOS PASOS:")
print(f"  1. Subir los 4 archivos a Kaggle para comparar resultados")
print(f"  2. Revisar 'data/analisis_ensemble.json' para más detalles")
print(f"  3. Considerar ajustar n_envios_ensemble según performance en Kaggle")

print("\n" + "="*60)


ENSAMBLE DE PREDICCIONES: ESTÁNDAR + SLOPE

ALINEANDO DATASETS POR NUMERO_DE_CLIENTE
Registros modelo estándar: 164,313
Registros modelo SLOPE: 164,313

✅ Registros después del merge: 164,313

MÉTODO 1: ENSAMBLE POR VOTACIÓN (UNIÓN)

📊 ESTADÍSTICAS DE VOTACIÓN:
  Predicciones modelo estándar (1): 14,002
  Predicciones modelo SLOPE (1): 14,251
  Predicciones UNIÓN (1): 17,569
  % del total: 10.69%

🔍 ANÁLISIS DE OVERLAP:
  Solo modelo estándar: 3,318
  Solo modelo SLOPE: 3,567
  Ambos modelos: 10,684
  Jaccard similarity: 60.81%

✅ Archivo votación guardado: 'data/predicciones_kaggle_ENSEMBLE_VOTACION_5semillas_20251022_1611.csv'

MÉTODO 2: ENSAMBLE POR PROMEDIO DE PROBABILIDADES
Probabilidades ensemble:
  Prob min: 0.000059
  Prob max: 0.953811
  Prob media: 0.014151

✅ Probabilidades ensemble guardadas

--- Opción 2A: Usar threshold en ensemble ---
Threshold ensemble: 0.025
Predicciones positivas: 14,725
% del total: 8.96%
✅ Archivo ensemble guardado: 'data/predicciones_kaggle_ENSEMB